In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
from matplotlib import pyplot as plt
import sys
import time

sys.path.append('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/')

from scripts.utils import create_one_hot_encoding

pd.set_option('display.max_columns', 500)

import warnings
warnings.filterwarnings("ignore")

In [2]:
df_transactions = pd.read_parquet('../../data/compressed_dataset/transactions.parquet')
# df_articles = pd.read_parquet('../../data/compressed_dataset/articles.parquet')
# df_customers = pd.read_parquet('../../data/compressed_dataset/customers.parquet')

# article_id_int = pd.read_pickle('../../data/compressed_dataset/article_id_int.pickle')
# int_article_id = pd.read_pickle('../../data/compressed_dataset/int_article_id.pickle')

# customer_id_int = pd.read_pickle('../../data/compressed_dataset/customer_id_int.pickle')
# int_customer_id = pd.read_pickle('../../data/compressed_dataset/int_customer_id.pickle')

# Days from last purchase

In [3]:
df_transactions = df_transactions[['t_dat', 'customer_id', 'article_id']]

In [4]:
df_transactions['t_dat'] = pd.to_datetime(df_transactions['t_dat'])
last_trans_date = df_transactions['t_dat'].max()


num_train_weeks = 20

for i in range(num_train_weeks-1, -1, -1):
    
    df_preds = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/ranker_train_set_utils/models_preds_{}.parquet'.format(i))
    
    last_local_date = last_trans_date - timedelta(days=i * 7)
    
    train_set_st1 = df_transactions[df_transactions['t_dat'] <= last_local_date].copy()
    train_set_st1['weeks_before_sub'] = i
    
    train_set_st1 = train_set_st1[train_set_st1['article_id'].isin(df_preds['article_id'])
                                  & train_set_st1['customer_id'].isin(df_preds['customer_id'])]
    
    df_trans_last = train_set_st1.groupby(['customer_id', 'article_id']).last().reset_index().rename(columns={'t_dat': 'last_dat'})
    
    train_set_st1 = df_preds.merge(df_trans_last[['customer_id', 'article_id', 'last_dat']],
                                   on=['customer_id', 'article_id'], how='left')
    
    train_set_st1['customer_article__num_days_from_last_purchase'] = (last_local_date - train_set_st1['last_dat']).dt.days
    
    train_set_st1 = train_set_st1[['customer_id', 'article_id', 'weeks_before_sub',
                                   'customer_article__num_days_from_last_purchase']]
    
    train_set_st1.fillna(-1, inplace=True)
    train_set_st1.to_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers_articles/dynamic/num_days_from_last_purchase/num_days_from_purchases_{}.parquet'.format(i),
                            index=False)    

OSError: Cannot save file into a non-existent directory: '/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers_articles/dynamic/num_days_from_last_purchase2'

In [5]:
pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers_articles/dynamic/num_days_from_last_purchase/num_days_from_purchases_1.parquet')

,customer_id,article_id,weeks_before_sub,customer_article__num_days_from_last_purchase
0,80,8,1,-1.0
1,80,67,1,-1.0
2,80,73,1,-1.0
3,80,1713,1,-1.0
4,80,1714,1,-1.0
...,...,...,...,...
6251217,1371960,104553,1,-1.0
6251218,1371960,104947,1,-1.0
6251219,1371960,105077,1,38.0
6251220,1371960,105078,1,-1.0


# Num purhases

In [3]:
df_transactions['t_dat'] = pd.to_datetime(df_transactions['t_dat'])
last_trans_date = df_transactions['t_dat'].max()


num_train_weeks = 1

for i in range(num_train_weeks-1, -1, -1):
    
    df_preds = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/ranker_train_set_utils/models_preds_{}.parquet'.format(i))
    
    last_local_date = last_trans_date - timedelta(days=i * 7)
    
    train_set_st1 = df_transactions[df_transactions['t_dat'] <= last_local_date].copy()
    train_set_st1['weeks_before_sub'] = i
    
    train_set_st1 = train_set_st1[train_set_st1['article_id'].isin(df_preds['article_id'])
                                  & train_set_st1['customer_id'].isin(df_preds['customer_id'])]
    
    
    train_set_st1_temp = df_preds.merge(train_set_st1.groupby(['customer_id', 'article_id'])['t_dat'].count().reset_index().rename(columns={'t_dat': 'customer_article__num_purchased'}),
                                        on=['customer_id', 'article_id'], how='left')
    
    train_set_st1_temp = train_set_st1_temp.merge(train_set_st1[train_set_st1['t_dat'] >= last_local_date-timedelta(days=90)].groupby(['customer_id', 'article_id'])['t_dat'].count().reset_index().rename(columns={'t_dat': 'customer_article__num_purchased_last90days'}),
                                        on=['customer_id', 'article_id'], how='left')
    
    train_set_st1_temp = train_set_st1_temp.merge(train_set_st1[train_set_st1['t_dat'] >= last_local_date-timedelta(days=30)].groupby(['customer_id', 'article_id'])['t_dat'].count().reset_index().rename(columns={'t_dat': 'customer_article__num_purchased_last30days'}),
                                        on=['customer_id', 'article_id'], how='left')
    
    train_set_st1_temp = train_set_st1_temp.merge(train_set_st1[train_set_st1['t_dat'] >= last_local_date-timedelta(days=7)].groupby(['customer_id', 'article_id'])['t_dat'].count().reset_index().rename(columns={'t_dat': 'customer_article__num_purchased_last7days'}),
                                        on=['customer_id', 'article_id'], how='left')
    
    train_set_st1_temp = train_set_st1_temp[['customer_id', 'article_id', 'weeks_before_sub',
                                             'customer_article__num_purchased',
                                             'customer_article__num_purchased_last90days',
                                             'customer_article__num_purchased_last30days',
                                             'customer_article__num_purchased_last7days']]
    
    train_set_st1_temp.fillna(0, inplace=True)
    
    train_set_st1_temp.to_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers_articles/dynamic/num_purchases/num_purchases_{}.parquet'.format(i),
                            index=False)
    